In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("spotify_millsongdata.csv")

In [3]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [5]:
df.shape

(57650, 4)

In [6]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [7]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [8]:
df.head(10)

,artist,song,text
0,Marilyn Manson,Unkillable Monster,How the fuck are we supposed to know \r\nWhen...
1,Howard Jones,ANGELS AND LOVERS,There's a whisper on the wind tonight \r\nIt ...
2,Nina Simone,Don't You Pay Them No Mind,People laugh each time they see us walking by ...
3,Z-Ro,Call My Phone,Bitch you aint gotta call my phone matter fact...
4,Hillsong United,Love Enough,Your love's enough to see the broken hearts \...
5,Out Of Eden,Lovely Day,"When troubles come my way, \r\nI look to you ..."
6,The Killers,Where The White Boys Dance,Take me to the place where the white boys danc...
7,Israel,Here I Am To Worship,Light of the world \r\nYou stepped down into ...
8,Our Lady Peace,Potato Girl,"We live, we die \r\nI don't know the reasons ..."
9,Pretenders,I Hurt You,I been crying like a woman \r\nBecause I'm ma...


In [9]:
df['text'][0]

'How the fuck are we supposed to know  \r\nWhen I\'m a monster, the way  \r\nYou refuse to die?  \r\nHow the fuck are we supposed to know  \r\nIf we\'re in love or if in we\'re pain  \r\n  \r\nI\'m a tightrope walker  \r\nCan\'t find my circus  \r\nAnd I\'m damaged beyond repair  \r\nNow you\'re just a coffin of a girl I knew  \r\nAnd I\'m buried in you  \r\n  \r\nYou never said "I\'ll end up like this"  \r\nOr "so"  \r\nYou never said "I\'ll end up like this"  \r\nNo  \r\n  \r\nSometimes I dream I\'m an exterminating angel  \r\nA traveling executioner from heaven  \r\nSent to give you the prettiest death I know  \r\nCall the grave and make our reservations  \r\n  \r\nYou never said "I\'ll end up like this"  \r\nYou never said "I\'ll end up like this"  \r\nNo,  \r\nYou never said "I\'ll end up like this"  \r\nYou never said "I\'ll end up like this"  \r\nNo,  \r\nYou never said "I\'ll end up like this"  \r\nYou never said "I\'ll end up like this"  \r\nNo  \r\n  \r\nAre we in love or are

In [10]:
# df = df.sample(5000)

In [11]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [12]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [13]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [14]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [17]:
similarity[0]

array([1.        , 0.08522252, 0.10089339, ..., 0.04484897, 0.01262159,
       0.10157932])

In [18]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [19]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [20]:
recommendation('Crying Over You')

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))